<a href="https://colab.research.google.com/github/holiday-scott/AppStoreAnalysis/blob/main/App_Store_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Most Profitable Free Apps on Google Play and Apple Store

#### Introduction
This is the first project in the Data Analyst in Python learning pathway with DataQuest.

#### Scenario
I am doing data analysis for a company that builds Android and iOS mobile apps. The company only makes apps that are free to download and install, so the main source of revenue consists of in-app ads. This means revenue for any given app is mostly influenced by the number of users who use the app — the more users that see and engage with the ads, the better. 

#### Goal
To understand what type of apps are likely to attract more users.

#### Dataset
* Apple Store: [This data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) contains more than 7000 Apple iOS mobile application details.
* Play Store: [This data set](https://www.kaggle.com/lava18/google-play-store-apps) contains more than 10000 mobile applicationd details.

## Part 1 - Prepration
First we mount Google Drive and read in the data sets.



In [ ]:
# Mount Google.
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
#drive.flush_and_unmount()

# Load reader modules
from csv import reader

# Find out where the data is.
#!ls "gdrive/MyDrive/Colab Notebooks/p1/"

### Google Play Data Set ###
opened_file = open('gdrive/MyDrive/Colab Notebooks/p1/PlayStore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

opened_file = open('gdrive/MyDrive/Colab Notebooks/p1/AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

# Show the first two lines of each data set
print("\n")
print("First two lines of the Apple Store data set")
print("-------------------------------------------")
print(ios_header)
print(ios[1])

print("First two lines of the Google Play Store data set")
print("-------------------------------------------------")
print(android_header)
print(android[1])

Mounted at /content/gdrive


First two lines of the Apple Store data set
-------------------------------------------
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']
First two lines of the Google Play Store data set
-------------------------------------------------
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


## Part 1 Continued
Next, we will need an inspection function to take a look at slices of the data as we clean it up.

In [ ]:
# Take data set, start and end point of slice, and count columns as a parameter
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line between rows
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

print(android_header)
print('\n')
explore_data(android, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


## Part 2 - Cleaning the Data

### Deleting Rows
This function shows a row and then asks if you want to deletes that row.

In [ ]:
### TEST LIST OF LISTS ###
#test_col = ["col0", "col1", "col2", "col3"]
#test_row1 = ["row1", "row1", "row1", "row1"]
#test_row2 = ["row2", "row2", "row2", "row2"]
#test_list_of_list = [test_col, test_row1, test_row2]

def del_row(data_set, row_index):
  print("This is row ", row_index)
  print(data_set[row_index])
  choice = input("Delete this row? [y/n]")
  if choice == "y": # only one valid answer
    del data_set[row_index]
    print("I have deleted row", row_index)
    #return data_set #use for testing.
  else: # else used so any other character results in no deletion 
    print("No action taken")
    #return data_set #use for testing.

#del_row(test_list_of_list,1)


We can now use this function to delete the known error on row **10472 of the Android data set**.

Since I'm not sure if the reported row counts to include the header row or not, we will use this function to inspect the row and delete if it matches the description: "Life Made WI-Fi..."

In [ ]:
#del_row(android, 10472)

The row was successfully deleted. Function commented out so it doesn't delete the next row by accident.

## Duplicate Data Points
Different versions of different apps, as well as updates, re-releases and scraping errors mean there may well be duplicate data points.

Analysis and visualisation will be helped by ensuring each app only occurs once in the data set.

The Android data set does indeed have multiple duplicates rows for the same application.

### Count Duplicates
This function counts the number of duplicate apps in the data set.

In [ ]:
### TEST LIST OF LISTS ###
#test_col = ["col 0", "col 1", "col 2", "col 3"]
#test_row1 = ["row 1", "row 1", "row 1", "row 1"]
#test_row2 = ["row 2", "row 2", "row 2", "row 2"]
#test_row3 = ["row 2", "row 2", "row 2", "row 2"]
#test = [test_col, test_row1, test_row2, test_row3]

def find_dup(dataset):
  duplicates = []
  uniques = []
  print("Total Rows:",len(dataset))
  for row in dataset:
    #print(len(duplicates)) # debug line
    #print(len(uniques)) # debug line
    name = row[0]
    #print(name) # debug line
    #print(row) # debug line
    if name in uniques:
      duplicates.append(name)
    else:
      uniques.append(name)
  print("Duplicate apps:", len(duplicates))
  print("Unique apps:", len(uniques))
  print("Checksum:", len(duplicates)+len(uniques))
  return

find_dup(android)



Total Rows: 10840
Duplicate apps: 1181
Unique apps: 9659
Checksum: 10840


### Delete Duplicates
Now we know they are there, we need to delete them.

In [ ]:
### TEST LIST OF LISTS ###
#test_col = ["col 0", "col 1", "col 2", "col 3"]
#test_row1 = ["row 1", "row 1", "row 1", "row 1"]
#test_row2 = ["row 2", "row 2", "row 2", "row 2"]
#test_row3 = ["row 2", "row 2", "row 2", "row 2"]
#test = [test_col, test_row1, test_row2, test_row3]

def del_dup(dataset):
  